In [ ]:
using Plots
#gr()
pyplot()
#plotly()

## A. Data Table choices

Machine learning is all about finding patterns in data, so it is very reasonble to start with data.

In [ ]:
# Some Data (try your own)
x = [5,6.5,7,8]
y = [10.1, 19.9, 30.1, 40.3]
# plot(x,y,
#     label="Y", line=(7,:green), marker=(10,0.8,:red), xlims=(0,10), ylims=(0,50),
#     xlabel="X",ylabel="Y")
    

### A.1. Just a matrix please. (No labels, no extras, simple.)

In [ ]:
data1 = [x y]

### A.2. Data Frames: Inspired by the R universe.

In [ ]:
using DataFrames
data2 = DataFrame(X=x,Y=y) # Upper Case X and Y are labels (not data)

In [ ]:
data2[1]

In [ ]:
#Pkg.add("CSV")
using CSV
CSV.write("data.csv", data2)

In [ ]:
;cat data.csv

### A.3. Indexed Tables (Treat data like array indices, knows type information)

In [ ]:
# Pkg.add("IndexedTables")
using  IndexedTables.Table
using IndexedTables
data3 = Table(Columns(X=x),Columns(Y=y))

In [ ]:
data3[6.5]

In [ ]:
typeof.([data1,data2,data3])

### A.4. JuliaDB (Lots of bells and whistles, many files, parallelism, ...)

In [ ]:
#Pkg.add("JuliaDB")
using JuliaDB:DTable
using JuliaDB

In [ ]:
data4 = distribute(data3, 1) 

In [ ]:
data5 = loadfiles(["data.csv"], usecache=false)

In [ ]:
typeof(data4)

In [ ]:
data4[1:2]

In [ ]:
select(data4,1=>i->i≥7) 

In [ ]:
filter(t->(t[1]>30),data4) 

### A.5 IterableTables

In [ ]:

#using IterableTables, DataTables, TypedTables # haven't investigated  much but looks very nice

## B. Simple Line Fitting

[So why is it called "Regression" anyway?](http://blog.minitab.com/blog/statistics-and-quality-data-analysis/so-why-is-it-called-regression-anyway) Dalton's original meaning not quite what it means today.

B.1 Linear Regression function

In [ ]:
b, w =  linreg(x,y)

In [ ]:
plot()
plot(x,y,
    label="Y", line=(4,:blue), marker=(3,0.8,:blue), xlims=(0,10), ylims=(0,50),
    xlabel="X",ylabel="Y")
plot!(x->w*x+b,xlims=(minimum(x)-.5,maximum(x)+.5), line=(4,:red), label="best fit line")
plot!(x->w*x+b, x ,marker=(3,0.8,:red), label="" )
for i = 1:length(x)
    plot!([x[i],x[i]],[y[i],w*x[i]+b],line=(4,:green))
end
plot!(legend=:topleft)

Mathematically equivalent Approaches <br>
B.2 Linear Algebra Least Squares

In [ ]:
A = [ones(x) x]

In [ ]:
A'A

In [ ]:
A\y 

In [ ]:
(A'A)\A'y  # normal equations usually not recommended

In [ ]:
q,r = qr(A)
r\(q'y)

In [ ]:
[length(x) sum(x); sum(x) x⋅x] \ [ sum(y) ; x⋅y ] # (A'A)\A'y

B.3 Basic Formula

In [ ]:
w = cov(x,y)/var(x) # same as (x.-mean(x))⋅(y.-mean(y))/sum(abs2,x.-mean(x))
b = mean(y)-w*mean(x)
b,w

In [ ]:
@which linreg(x,y) # essentially uses the above formula

B.4 optimization  (think machine learning) via the package optim.jl

In [ ]:
using Optim   # Julia all the way down
loss(bw) = sum(abs2,bw[2]*x.+bw[1]-y) # uglyish
optimize(loss,[0.0,0.0]).minimizer

B.5 optimization with the package JuMP <br>
Note not every julia function can be in @objective or @NLobjective
but that would be the goal. See  [linear and quadratic objective Jump Notes](http://www.juliaopt.org/JuMP.jl/0.18/refexpr.html)  and [Nonlinear Jump Notes](http://www.juliaopt.org/JuMP.jl/0.18/nlp.html#syntax-notes).

In [ ]:
#Pkg.add("Ipopt")

In [ ]:
using JuMP, Ipopt
n = length(x)
m = Model(solver=IpoptSolver(print_level=0))
@variable(m,w)
@variable(m,b)
@objective(m, Min, sum((w*x[i]+b-y[i])^2 for i in 1:n))
#@objective(m, Min,   sum(abs2,  w*x+b-y))
solve(m)
println( " b = ", getvalue(b), "w = ", getvalue(w))

B.6 Generalized Linear Models <br>
the very fancy statistical thing

In [ ]:
#Pkg.add("GLM")
using GLM # Generalized Linear Models

In [ ]:
lm(@formula(Y~X), data2)

The lines above are obviously b and w
We assume at the start X is known without error, b,w,σ are unknown and
the real Y is distributed like  b+w*X+$\sigma *$randn(),
and the Y we have are samples from this distribution.

Under these assumptions, if we fit many times, the b and w would be normal, with these predicted standard deviations.

The third column is just the ratio of column 1 to column 2 , thus normalizing the situation to a standard normal.

When the probability column is less than .05, we can reject the hypothesis that the intercept/slope is 0 at the 5 percent signficance level. What does this mean? It means we feel pretty good about our intercept and slope. If the probability is higher than .05 we can not reject the null hypothesis, meaning that we feel 0 for the intercept/slope could have been possible. In particular a 0 slope says that the dependent variable is not really statistically dependent after all.

### C. Stochastic Gradient Descent

In [ ]:
loss(w,b,i) =(w*x[i]+b-y[i])^2  # loss due to point i
Dloss(w,b,i) = 2*(w*x[i]+b-y[i])*[x[i];1]

In [ ]:
w,b = 0.0, 0.0
for t=1:100000
    η = .002  # there seems to be an art to picking these steplengths
    i = rand(1:4)
    d = Dloss(w,b,i)
    w -= η * d[1]
    b -= η * d[2]  
end
 println(b," ",w)   

In [ ]:
loss(w,b,i) =(w*χ[i]+b-y[i])^2  # loss due to point i
Dloss(w,b,i) = 2*(w*χ[i]+b-y[i])*[χ[i];1]

In [ ]:
μ = mean(x)
σ = std(x)
χ = (x-μ)/σ

w,b = 0.0, 0.0
for t=1:100000
    η = .01  # there seems to be an art to picking these steplengths
    i = rand(1:4)
    d = Dloss(w,b,i)
     w -= η * d[1]
     b -= η * d[2] 
    ## instead fancy update rules like Adam ??
   
end
 println(b-w*μ/σ," ",w/σ)


###  D. KNET

In [ ]:
#Pkg.add("Knet")
using Knet

In [ ]:
predict(w,x) = w[2]*x .+ w[1]
loss(w,x,y) = sum(abs2, y - predict(w,x)) 

In [ ]:
lossgradient = grad(loss)

In [ ]:
function train(w, data; lr=.1)
    p=1
    for (x,y) in data
        println("This is pass $p")
        p+=1
        dw = lossgradient(w, x, y)
        for i in 1:length(w)
            w[i] -= lr * dw[i]
        end
    end
    return w
end

In [ ]:
train([0.0,0.0],zip(x,y),lr=.01) # not enough data

In [ ]:
data = [(x[i],y[i]) for i=1:4]

In [ ]:
function train2(w, data; lr=.1)
       for t in 1:10000
          
        (x,y) = data[rand(1:4)]
        dw = lossgradient(w, x, y)
            for i=1:length(w)
            w[i] -= lr * dw[i]
            #update(w, lossgradient(w,x,y), adam())
        end
    end
    return w
end

In [ ]:
train2([0.0;0.0],data,lr=.01) 

### E. TensorFlow

Jon Malmaud explains why Julia+TensorFlow is way better than Python+Tensorflow
[https://www.youtube.com/watch?v=MaCf1PtHEJo](https://www.youtube.com/watch?v=MaCf1PtHEJo)

Why use Julia API when you can use Python API? <br>
At 3:20 he explains three reasons: <br>
1. Julia's Multiple Dispatch System <br>
2. Julia's macro system <br>
3. Julia's JIT compiler

In [1]:
x = [5,6.5,7,8]
y = [10.1, 19.9, 30.1, 40.3]

4-element Array{Float64,1}:
 10.1
 19.9
 30.1
 40.3

In [2]:
using TensorFlow

In [ ]:
session=Session()

In [ ]:
@tf X=placeholder(Float32)

In [ ]:
@tf W=get_variable([], Float32)

In [ ]:

@tf b=get_variable([], Float32)

In [ ]:
@tf Y=X.*W + b

In [ ]:
@tf Y_obs=placeholder(Float32)

In [ ]:
@tf Loss=reduce_sum((Y.-Y_obs).^2)

In [ ]:
optimizer=train.GradientDescentOptimizer(1e-3)

In [ ]:
minimizer=train.minimize(optimizer, Loss)

In [ ]:
run(session, global_variables_initializer())
for i in 1:20000
    run(session, minimizer, Dict(X=>[5,6.5,7,8], Y_obs=>[10.1,19.9,30.1,40.3]))
end

In [ ]:
run(session, [b, W])

In [ ]:
visualize()

In [ ]:
Pkg.update()